In [1]:
#Step 1: load data.

In [2]:
#Step 1a: load data from ACS 
#using censusdata package and documentation 
#from here: https://github.com/jtleider/censusdata

In [3]:
import sys
sys.path.append('..')

In [4]:
from src.data.censusdata import censusdata

In [5]:
import pandas as pd

In [6]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [7]:
#Search for table and variable combinations for desired stats.

In [8]:
acs_version = 'acs5' #or 'acs5'

In [9]:
#search for race-related variables
#output supressed.

#censusdata.search(acs_version, 2018, 'label', 'white')

In [10]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B02001'))

In [11]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B03002'))

In [12]:
#White, not Hispanic: B03002_003E (from total: B03002_001E)
#White, Hispanic: B03002_013E (from total: B03002_001E)
#Black: B02001_003E (from total: B02001_001E)
#Other: If totals are equal: (B02001_001E and B03002_001E) then other is total minus 
#White, not Hispanic plus White, Hispanic plus Black.

#Will check it out after finding geography codes.

In [13]:
#search for Socioeconomic status-related variables
#censusdata.search(acs_version, 2018, 'label', 'degree')

In [14]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B15003'))

In [15]:
#censusdata.search(acs_version, 2018, 'label', 'per capita income')

In [16]:
#censusdata.printtable(censusdata.censustable(acs_version, 2018, 'B19301'))

In [17]:
#Finding geography codes for all counties in Texas

In [18]:
#output supressed.
#censusdata.geographies(censusdata.censusgeo([('state', '*')]), acs_version, 2018)
#Texas is '48'
#censusdata.geographies(censusdata.censusgeo([('state', '48'), ('county', '*')]), acs_version, 2018)
#Travis County is '453' (used for testing)

In [19]:
txcounties = censusdata.download(acs_version, 2018,
                             censusdata.censusgeo([('state', '48'), ('county', '*')]),
                             ['B03002_001E','B02001_001E','B03002_003E','B03002_013E','B02001_003E', \
                              'B15003_022E','B15003_001E', \
                              'B19301_001E'])


In [20]:
#Check that total from the two tables are the same for all counties in Texas.
assert (txcounties.B03002_001E == txcounties.B02001_001E).all()

In [21]:
txcounties['population_2018'] = txcounties.B03002_001E
txcounties['fraction_white_nh'] = txcounties.B03002_003E / txcounties.B03002_001E
txcounties['fraction_hispanic'] = txcounties.B03002_013E / txcounties.B03002_001E
txcounties['fraction_black'] = txcounties.B02001_003E / txcounties.B02001_001E
txcounties['fraction_other_race'] = 1. - txcounties['fraction_white_nh'] - \
                                txcounties['fraction_hispanic'] - txcounties['fraction_black']

txcounties['fraction_w4yrdeg'] = txcounties.B15003_022E / txcounties.B15003_001E
txcounties['per_capita_income'] = txcounties.B19301_001E


In [22]:
txcounties = txcounties[['population_2018', 'fraction_white_nh', 'fraction_hispanic', \
                         'fraction_black','fraction_other_race', \
                         'fraction_w4yrdeg','per_capita_income']]
txcounties.describe()

,population_2018,fraction_white_nh,fraction_hispanic,fraction_black,fraction_other_race,fraction_w4yrdeg,per_capita_income
count,2.54e+02,2.54e+02,254.00,254.00,2.54e+02,254.00,254.00
mean,1.10e+05,5.61e-01,0.28,0.06,9.76e-02,0.13,25497.25
std,3.96e+05,2.12e-01,0.22,0.07,8.06e-02,0.05,5292.63
min,1.02e+02,7.28e-03,0.02,0.00,2.92e-03,0.00,13350.00
25%,7.10e+03,4.19e-01,0.13,0.01,4.80e-02,0.10,22265.50
50%,1.86e+04,5.88e-01,0.20,0.04,7.46e-02,0.12,25128.50
75%,4.99e+04,7.31e-01,0.38,0.09,1.20e-01,0.15,28453.75
max,4.60e+06,9.07e-01,0.94,0.34,5.83e-01,0.33,43439.00


In [23]:
#add column of county number (as state number and county number)

county_numbers = []
for value in txcounties.index.values:
    county_numbers.append(int('48'+value.geo[1][1]))

In [24]:
txcounties['county_code'] = county_numbers
txcounties = txcounties.set_index('county_code')

In [25]:
#load religiosity data.
import pandas as pd

In [35]:
religion = pd.read_excel('../data/external/U.S. Religion Census Religious '+\
                         'Congregations and Membership Study, 2010 (County File).XLSX')

In [36]:
#pull columns for:
#Evangelical
#Catholic
#Mormon
#Total population in 2010
#County Code
religion = religion[religion['STABBR']=='TX'].loc[:, \
          ['EVANADH','CATHADH','LDSADH','POP2010','CNTYCODE','CNTYNAME']]

In [37]:
religion['county_code'] = religion['CNTYCODE']+48000

In [38]:
religion['County'] = religion['CNTYNAME'].str.split('\s').str[0]
religion['County'] = religion['County'].str.upper()

In [39]:
#convert number to fraction
religion['fraction_evangelical'] = religion['EVANADH'] / religion['POP2010']
religion['fraction_catholic'] = religion['CATHADH'] / religion['POP2010']
religion['fraction_mormon'] = religion['LDSADH'] / religion['POP2010']
religion = religion.fillna(0)
religion['fraction_other_religion'] = 1. - religion['fraction_evangelical'] \
                            - religion['fraction_catholic'] - religion['fraction_mormon']

In [40]:
religion = religion[['county_code','fraction_evangelical', 'fraction_catholic', \
                     'fraction_mormon','fraction_other_religion','County']]
religion.describe()

,county_code,fraction_evangelical,fraction_catholic,fraction_mormon,fraction_other_religion
count,254.00,254.00,254.00,2.54e+02,254.00
mean,48254.00,0.35,0.17,6.86e-03,0.47
std,146.94,0.17,0.15,7.79e-03,0.16
min,48001.00,0.00,0.00,0.00e+00,-0.04
25%,48127.50,0.22,0.06,0.00e+00,0.39
50%,48254.00,0.34,0.12,5.23e-03,0.50
75%,48380.50,0.46,0.23,1.28e-02,0.59
max,48507.00,0.90,0.91,3.83e-02,0.93


In [41]:
#Find which counties have a negative percent other
import numpy as np
religion[religion['fraction_other_religion'] < 0].index

Int64Index([2591], dtype='int64')

In [42]:
#Set percent other to 0 for one county with a negative percent other
religion.at[2591,'fraction_other_religion'] = 0

In [43]:
religion.describe()

,county_code,fraction_evangelical,fraction_catholic,fraction_mormon,fraction_other_religion
count,254.00,254.00,254.00,2.54e+02,254.00
mean,48254.00,0.35,0.17,6.86e-03,0.48
std,146.94,0.17,0.15,7.79e-03,0.16
min,48001.00,0.00,0.00,0.00e+00,0.00
25%,48127.50,0.22,0.06,0.00e+00,0.39
50%,48254.00,0.34,0.12,5.23e-03,0.50
75%,48380.50,0.46,0.23,1.28e-02,0.59
max,48507.00,0.90,0.91,3.83e-02,0.93


In [44]:
religion = religion.set_index('county_code')

In [45]:
#let's get the land area of each county
land_area = pd.read_csv('https://query.data.world/s/bblueltx7fslj2s4gu65nyv5bcohuq')

In [46]:
#Load results from 2016 presidential election to get partisan lean
partisan_lean = pd.read_csv('https://query.data.world/s/5pczct6v6ltnjjz5m7lodlcetar5mh')

In [47]:
partisan_lean['margin_of_victory'] = partisan_lean['dPct'] - partisan_lean['rPct']

In [48]:
#land_area.loc[:,['County','TotalPopulation','LandAreaSqMiles'] \
#             ][land_area['County']==48261]

In [49]:
land_area['county_code'] = land_area['County']
land_area['margin_of_victory_D2016'] = partisan_lean['margin_of_victory']

In [50]:
land_area = land_area[['county_code','LandAreaSqMiles','margin_of_victory_D2016']]
land_area = land_area[(land_area['county_code'] > 48000) & (land_area['county_code'] < 49000)]

In [51]:
land_area = land_area.set_index('county_code')

In [52]:
land_area

,LandAreaSqMiles,margin_of_victory_D2016
county_code,,
48001,1062.60,-0.58
48003,1500.72,-0.63
48005,797.78,-0.47
48007,252.08,-0.50
48009,903.11,-0.79
...,...,...
48499,645.24,-0.70
48501,799.72,-0.60
48503,914.47,-0.74


In [53]:
all_features = pd.merge(txcounties, religion, on='county_code')
all_features = pd.merge(all_features, land_area, on='county_code')

In [54]:
all_features['population_density'] = all_features['population_2018'] \
/ all_features['LandAreaSqMiles']

In [55]:
all_features

,population_2018,fraction_white_nh,fraction_hispanic,fraction_black,fraction_other_race,fraction_w4yrdeg,per_capita_income,fraction_evangelical,fraction_catholic,fraction_mormon,fraction_other_religion,County,LandAreaSqMiles,margin_of_victory_D2016,population_density
county_code,,,,,,,,,,,,,,,
48015,29565,0.63,0.18,0.09,0.11,0.17,30858,0.17,0.29,1.73e-02,0.51,AUSTIN,646.49,-0.60,45.73
48261,595,0.12,0.84,0.00,0.04,0.03,15820,0.00,0.30,0.00e+00,0.70,KENEDY,1458.45,0.08,0.41
48355,360486,0.30,0.60,0.04,0.06,0.14,27649,0.20,0.33,1.14e-02,0.46,NUECES,838.31,-0.02,430.01
48089,21022,0.56,0.17,0.13,0.14,0.14,27861,0.14,0.31,0.00e+00,0.55,COLORADO,960.29,-0.51,21.89
48409,67046,0.39,0.54,0.02,0.06,0.10,25281,0.22,0.32,1.23e-02,0.45,SAN,693.44,-0.24,96.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48307,8098,0.61,0.23,0.02,0.15,0.13,24409,0.32,0.13,1.40e-02,0.53,MCCULLOCH,1065.60,-0.67,7.60
48287,16952,0.64,0.16,0.11,0.09,0.11,25876,0.42,0.11,0.00e+00,0.47,LEE,629.04,-0.55,26.95
48139,168838,0.62,0.20,0.10,0.09,0.16,30727,0.32,0.12,7.67e-03,0.56,ELLIS,935.66,-0.45,180.45


In [56]:
#read in and load 2016 and 2020 Texas Democratic Primary results
results2020 = pd.read_csv('../data/external/Texas Primary Democratic Party Results 2020 - County.csv', \
           delimiter='\t', thousands=',')

In [57]:
results2020 = results2020[results2020.index > 0]

In [58]:
results2020['fraction_sanders_2020'] = results2020['SANDERS'].astype(float)/ \
 results2020['Total_Votes'].astype(float)

In [59]:
results2020 = results2020[['County','fraction_sanders_2020']]

In [60]:
results2020

,County,fraction_sanders_2020
1,ANDERSON,0.21
2,ANDREWS,0.36
3,ANGELINA,0.20
4,ARANSAS,0.19
5,ARCHER,0.17
...,...,...
250,WOOD,0.17
251,YOAKUM,0.34
252,YOUNG,0.19
253,ZAPATA,0.26


In [61]:
results2016 = pd.read_csv('../data/external/Texas Primary 2016 Results by County.csv', \
                          thousands=',')

In [62]:
results2016['Total_Votes'] = results2016['Clinton_Hillary']
other_dem_names = ['De La Fuente_Rocky', 'Hawes_Calvis', 'Judd_Keith', 'Locke_Star', \
                   "O'Malley_Martin", "Sanders_Bernie", "Wilson_Willie"]
for dem_name in other_dem_names:
    results2016['Total_Votes'] = results2016['Total_Votes'] + results2016[dem_name]




In [63]:
results2016['fraction_sanders_2016'] = results2016['Sanders_Bernie'].astype(float)/ \
 results2016['Total_Votes'].astype(float)

In [64]:
results2016 = results2016[['County','fraction_sanders_2016']]

In [65]:
results2016

,County,fraction_sanders_2016
0,Anderson,0.23
1,Andrews,0.33
2,Angelina,0.24
3,Aransas,0.35
4,Archer,0.38
...,...,...
249,Wood,0.30
250,Yoakum,0.48
251,Young,0.39
252,Zapata,0.24


In [66]:
#combine results dataframes on County.
#Capitalize County in results2016.
results2016['County'] = results2016['County'].str.upper()

In [67]:
results = pd.merge(results2016, results2020, on='County')

In [68]:
results

,County,fraction_sanders_2016,fraction_sanders_2020
0,ANDERSON,0.23,0.21
1,ANDREWS,0.33,0.36
2,ANGELINA,0.24,0.20
3,ARANSAS,0.35,0.19
4,ARCHER,0.38,0.17
...,...,...,...
249,WOOD,0.30,0.17
250,YOAKUM,0.48,0.34
251,YOUNG,0.39,0.19
252,ZAPATA,0.24,0.26


In [69]:
all_features = pd.merge(all_features, results, on='County')

In [70]:
all_features

,population_2018,fraction_white_nh,fraction_hispanic,fraction_black,fraction_other_race,fraction_w4yrdeg,per_capita_income,fraction_evangelical,fraction_catholic,fraction_mormon,fraction_other_religion,County,LandAreaSqMiles,margin_of_victory_D2016,population_density,fraction_sanders_2016,fraction_sanders_2020
0,29565,0.63,0.18,0.09,0.11,0.17,30858,0.17,0.29,1.73e-02,0.51,AUSTIN,646.49,-0.60,45.73,0.25,0.21
1,595,0.12,0.84,0.00,0.04,0.03,15820,0.00,0.30,0.00e+00,0.70,KENEDY,1458.45,0.08,0.41,0.30,0.18
2,360486,0.30,0.60,0.04,0.06,0.14,27649,0.20,0.33,1.14e-02,0.46,NUECES,838.31,-0.02,430.01,0.28,0.25
3,21022,0.56,0.17,0.13,0.14,0.14,27861,0.14,0.31,0.00e+00,0.55,COLORADO,960.29,-0.51,21.89,0.24,0.18
4,11473,0.86,0.05,0.03,0.07,0.08,26165,0.32,0.06,0.00e+00,0.62,RAINS,229.45,-0.71,50.00,0.33,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,8098,0.61,0.23,0.02,0.15,0.13,24409,0.32,0.13,1.40e-02,0.53,MCCULLOCH,1065.60,-0.67,7.60,0.30,0.20
233,16952,0.64,0.16,0.11,0.09,0.11,25876,0.42,0.11,0.00e+00,0.47,LEE,629.04,-0.55,26.95,0.32,0.22
234,168838,0.62,0.20,0.10,0.09,0.16,30727,0.32,0.12,7.67e-03,0.56,ELLIS,935.66,-0.45,180.45,0.29,0.24
235,51365,0.69,0.20,0.02,0.09,0.17,30498,0.25,0.11,1.50e-02,0.62,KERR,1103.33,-0.56,46.55,0.41,0.22


In [74]:
all_features.to_csv('../data/interim/all_features.csv', index=False)